# Dashboard

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, html, dcc
from statsmodels.tsa.seasonal import seasonal_decompose

# Загрузка данных
deals_df = pd.read_pickle('deals_cleaned.pkl')
contacts_df = pd.read_pickle('contacts_cleaned.pkl')
spend_df = pd.read_pickle('spend_cleaned.pkl')
metrics_df = pd.read_pickle('metrics.pkl')

# Метрики
cm = metrics_df.loc[metrics_df['Metric'] == 'CM', 'Value'].values[0] # -- маржинальная прибыль 
c1 = metrics_df.loc[metrics_df['Metric'] == 'C1', 'Value'].values[0] # -- конверсия из лида в клиента -- 
cltv_row = metrics_df.loc[metrics_df['Metric'] == 'CLTV', 'Value'].values[0] # -- средняя валовая прибыль на клиента --
if isinstance(cltv_row, str):
    cltv = float(cltv_row.replace(',', ''))
else:
    cltv = float(cltv_row)
    
leads = contacts_df['Id'].nunique()
contact_all_deals = deals_df['Contact Name'].nunique()
success_contacts = deals_df[(deals_df['Stage'] == 'Payment Done') & (deals_df['Offer Total Amount'].notna())]['Contact Name'].nunique()
success_deals = deals_df[(deals_df['Stage'] == 'Payment Done') & (deals_df['Offer Total Amount'].notna())]['Id'].nunique()
all_deals = deals_df['Id'].nunique()
aov_value = metrics_df.loc[metrics_df['Metric'] == 'AOV', 'Value'].values[0]
# ltc_value = metrics_df.loc[metrics_df['Metric'] == 'LTC (CPA)', 'Value'].values[0]

def format_num(num):
    if isinstance(num, str):
        num = num.replace(',', '') 
    num = float(num)
    if abs(num) >= 1_000_000_000:
        return f"{num/1_000_000_000:.1f}B"
    elif abs(num) >= 1_000_000:
        return f"{num/1_000_000:.1f}M"
    elif abs(num) >= 1_000:
        return f"{num/1_000:.1f}K"
    else:
        return f"{num:.0f}"

# --- Группировки для воронки и heatmap ---
grouped_deals = deals_df.groupby('Source')
leads_by_source = grouped_deals['Contact Name'].nunique().rename('Leads')
total_deals_by_source = grouped_deals['Id'].count().rename('Deals')
successful_deals_by_source = deals_df[(deals_df['Stage'] == 'Payment Done') & (deals_df['Offer Total Amount'].notna())].groupby('Source')['Id'].count().rename('Successful Deals')

funnel_df = pd.concat([leads_by_source, total_deals_by_source, successful_deals_by_source], axis=1).fillna(0)
funnel_df = funnel_df.astype({'Leads': int, 'Deals': int, 'Successful Deals': int}).reset_index()
funnel_df['Conversion'] = (funnel_df['Successful Deals'] / funnel_df['Deals']) * 100

# source_stats для heatmap
source_stats = funnel_df[['Source', 'Deals', 'Successful Deals']].copy()
source_stats = source_stats.rename(columns={'Deals': 'total_deals', 'Successful Deals': 'successful_deals'})
source_stats['conversion_rate'] = (source_stats['successful_deals'] / source_stats['total_deals']) * 100
source_stats = source_stats[(source_stats['total_deals'] > 0) & (source_stats['conversion_rate'] > 0)]
source_stats = source_stats.set_index('Source')

# --- Time series (динамика и тренд) ---
deals_df = deals_df.sort_values('Created Time')
deals_df.set_index('Created Time', inplace=True)
daily_deals = deals_df.resample("D").size()
decomposition = seasonal_decompose(daily_deals, model='additive', period=7)
trend = daily_deals.rolling(window=30, center=True).mean() 

fig_timeseries = go.Figure()
fig_timeseries.add_trace(go.Scatter(
    x=daily_deals.index, y=daily_deals,
    name='Dynamics',
    line=dict(color='royalblue')
))
fig_timeseries.add_trace(go.Scatter(
    x=trend.index, y=trend,
    name='Trend',
    line=dict(color='firebrick', width=3)
))
fig_timeseries.update_layout(
    title={'text': 'Dynamics and trend for successful deals by days', 'font': {'size': 24}, 'x': 0.5, 'xanchor': 'center'},
    xaxis_title='Date',
    yaxis_title='Count',
    legend=dict(x=0.01, y=0.99),
    height=500
)
fig_timeseries.update_xaxes(
    tickformat="%b %Y",
    dtick="M1",
    tickangle=45
)

# --- Heatmap ---
heatmap_data = source_stats[['total_deals', 'successful_deals', 'conversion_rate']]

# Переименуем столбцы для красивого отображения
heatmap_data = heatmap_data.rename(columns={
    'total_deals': 'Total Deals',
    'successful_deals': 'Successful Deals',
    'conversion_rate': 'Conversion Rate'
})

heatmap_norm = heatmap_data.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
heatmap_norm.columns = heatmap_data.columns  # чтобы индексы совпадали


def get_font_color(norm_value, threshold=0.6):
    return 'white' if norm_value > threshold else 'black'

annotations = []
for n, row in enumerate(heatmap_data.index):
    for m, col in enumerate(heatmap_data.columns):
        value = heatmap_data.iloc[n, m]
        norm_value = heatmap_norm.iloc[n, m]
        font_color = get_font_color(norm_value, threshold=0.6)
        annotations.append(
            dict(
                x=col,
                y=row,
                text=str(round(value, 1)),
                showarrow=False,
                font=dict(color=font_color, size=12)
            )
        )

heatmap_fig = go.Figure(
    data=go.Heatmap(
        z=heatmap_norm.values,
        x=heatmap_norm.columns,
        y=heatmap_norm.index,
        colorscale='YlGnBu',
        showscale=True,
        hovertemplate='Source: %{y}<br>%{x}: %{z:.2f}<extra></extra>'
    )
)
heatmap_fig.update_layout(
    title={'text': 'Marketing sources effectiveness', 'font': {'size': 24}, 'x': 0.5, 'xanchor': 'center'},
    annotations=annotations,
    xaxis_title='Indicator',
    yaxis_title='Source',
    margin=dict(l=100, r=20, t=50, b=50),
    height=40*len(heatmap_data)+250,  # увеличили высоту
    width=650 
)

# --- Campaigns comparison ---
deals_with_contacts = deals_df[['Campaign', 'Contact Name']].dropna()
leads_per_campaign = deals_with_contacts.groupby('Campaign', observed=True)['Contact Name'].nunique().reset_index()
leads_per_campaign.columns = ['Campaign', 'Leads']
impressions_per_campaign = spend_df.groupby('Campaign', observed=True, as_index=False)['Impressions'].sum()
merged_df = impressions_per_campaign.merge(leads_per_campaign, on='Campaign', how='left')
merged_df['Leads'] = merged_df['Leads'].fillna(0)
merged_df['Conversion Rate (%)'] = merged_df.apply(
    lambda row: (row['Leads'] / row['Impressions'] * 100) if row['Impressions'] > 0 else None,
    axis=1
)
merged_df_sorted = merged_df.sort_values('Conversion Rate (%)', ascending=False)
top10_by_conversion = merged_df_sorted.head(10)

campaigns_fig = go.Figure()
campaigns_fig.add_trace(go.Bar(
    x=top10_by_conversion['Campaign'],
    y=top10_by_conversion['Impressions'],
    name='Impressions (reach)',
    marker_color='skyblue',
    yaxis='y1',
    text=top10_by_conversion['Impressions'],           # <-- подписи
    textposition='outside',                            # <-- положение над столбиком
    texttemplate='%{text:,}'
))
campaigns_fig.add_trace(go.Scatter(
    x=top10_by_conversion['Campaign'],
    y=top10_by_conversion['Conversion Rate (%)'],
    name='Conversion Rate (%)',
    mode='lines+markers',
    marker=dict(color='red'),
    line=dict(color='red', width=3),
    yaxis='y2'
))
campaigns_fig.update_layout(
    title={'text':'Comparison of campaigns by reach and conversion for all deals (Top 10)', 'font': {'size': 24}, 'x': 0.5, 'xanchor': 'center'},
    xaxis=dict(title='Campaign', tickangle=45),
    yaxis=dict(title=dict(text='Impressions (reach)', font=dict(color='skyblue')), tickfont=dict(color='skyblue')),
    yaxis2=dict(title=dict(text='Conversion Rate (%)', font=dict(color='red')), tickfont=dict(color='red'), overlaying='y', side='right'),
    legend=dict(x=0.01, y=0.99),
    bargap=0.3,
    height=500,
    showlegend=False
)
# Найти минимальное и максимальное значение и их индексы
min_idx = top10_by_conversion['Conversion Rate (%)'].idxmin()
max_idx = top10_by_conversion['Conversion Rate (%)'].idxmax()

min_x = top10_by_conversion.loc[min_idx, 'Campaign']
min_y = top10_by_conversion.loc[min_idx, 'Conversion Rate (%)']

max_x = top10_by_conversion.loc[max_idx, 'Campaign']
max_y = top10_by_conversion.loc[max_idx, 'Conversion Rate (%)']

# Добавить аннотации
campaigns_fig.add_annotation(
    x=min_x,
    y=min_y,
    yref='y2',
    text=f"Min: {min_y:.2f}%",
    showarrow=True,
    arrowhead=2,
    ax=0,
    ay=40,
    font=dict(color="red", size=12),
    bgcolor="white"
)
campaigns_fig.add_annotation(
    x=max_x,
    y=max_y,
    yref='y2',
    text=f"Max: {max_y:.2f}%",
    showarrow=True,
    arrowhead=2,
    ax=0,
    ay=-40,
    font=dict(color="green", size=12),
    bgcolor="white"
)

# --- Map ---
successful_deals_df = deals_df[(deals_df['Stage'] == 'Payment Done') & (deals_df['Offer Total Amount'].notna())]
map_df = successful_deals_df.dropna(subset=['Latitude', 'Longitude'])
fig_map = px.scatter_map(
    map_df,
    lat='Latitude',
    lon='Longitude',
    hover_name='City',
    hover_data=['City', 'German level'],
    zoom=3,
    height=600
)
fig_map.update_layout(
    mapbox_style="satellite-streets",
    margin={"r":0,"t":0,"l":0,"b":0}
)

# --- Dash app ---
app = Dash(__name__)

metric_box_style = {
    'background': '#f8f9fa',
    'borderRadius': '10px',
    'padding': '10px 20px',
    'boxShadow': '0 2px 6px rgba(0,0,0,0.05)',
    'display': 'flex',
    'flexDirection': 'column',
    'alignItems': 'center',
    'minWidth': '120px'
}

app.layout = html.Div([
    html.H1("General analytical dashboard", style={
        'paddingLeft': '0', 'marginLeft': '0',
        'textAlign': 'center',
        'fontWeight': 'bold',
        'fontSize': '48px',
        'marginBottom': '20px'
    }),
    html.Div([
        html.Div([
            dcc.Graph(figure=go.Figure(go.Funnel(
                y=["Leads", "Opportunities", "Buyers"],
                x=[leads, contact_all_deals, success_contacts],
                textinfo="value+percent initial",
                texttemplate='%{value:,.0f} (%{percentInitial:.1%})'  # Явно указываем формат без K/M/B
                )).update_layout(
                title={'text': "Sales funnel", 'font': {'size': 24}, 'x': 0.5, 'xanchor': 'center'},
                margin=dict(t=40, b=40),
                height=400,
                width=600
            ), style={
                'height': '400px',
                'width': '600px',
                'margin': '0 0 30px 0'
            }),
            dcc.Graph(figure=heatmap_fig, style={
                'height': f'{40*len(heatmap_data)+100}px',
                'width': '700px',
                'maxWidth': '100%'
            })
        ], style={
            'display': 'flex',
            'flexDirection': 'column',
            'alignItems': 'center',  
            'width': '700px',
            'marginRight': '30px',
            'marginLeft': '0',
            'justifyContent': 'flex-start'
        }),
        html.Div([
            html.Div([
                html.Div([
                    html.H4("CM", style={'margin': '0', 'fontSize': '20px', 'fontWeight': 'bold'}),
                    html.P(format_num(cm), style={'fontSize': '38px', 'fontWeight': 'bold', 'margin': '4px 0 0 0'})
                ], style=metric_box_style),
                html.Div([
                    html.H4("AOV", style={'margin': '0', 'fontSize': '20px', 'fontWeight': 'bold'}),
                    html.P(f"{aov_value}", style={'fontSize': '38px', 'fontWeight': 'bold', 'margin': '4px 0 0 0'})
                ], style=metric_box_style),
                html.Div([
                    html.H4("CLTV", style={'margin': '0', 'fontSize': '20px', 'fontWeight': 'bold'}),
                    html.P(f"{cltv}", style={'fontSize': '38px', 'fontWeight': 'bold', 'margin': '4px 0 0 0'})
                ], style=metric_box_style),
                html.Div([
                    html.H4("Conversion", style={'margin': '0', 'fontSize': '20px', 'fontWeight': 'bold'}),
                    html.P(f"{c1}", style={'fontSize': '38px', 'fontWeight': 'bold', 'margin': '4px 0 0 0'})
                ], style=metric_box_style),
                html.Div([
                    html.H4("Deals", style={'margin': '0', 'fontSize': '20px', 'fontWeight': 'bold'}),
                    html.P(f"{all_deals}", style={'fontSize': '38px', 'fontWeight': 'bold', 'margin': '4px 0 0 0'})
                ], style=metric_box_style),
                html.Div([
                    html.H4("Closed deals", style={'margin': '0', 'fontSize': '20px', 'fontWeight': 'bold'}),
                    html.P(f"{success_deals}", style={'fontSize': '38px', 'fontWeight': 'bold', 'margin': '4px 0 0 0'})
                ], style=metric_box_style),
            ], style={
                'display': 'flex',
                'gap': '20px',
                'justifyContent': 'flex-start',
                'flexWrap': 'nowrap',
                'marginBottom': '20px'
            }),
            dcc.Graph(figure=fig_timeseries, style={
                'height': '500px',
                'width': '1200px',
                'maxWidth': '100%',
                'marginTop': '0'
            }),
            dcc.Graph(figure=campaigns_fig, style={
                'height': '600px',
                'width': '1200px',
                'maxWidth': '100%',
                'marginTop': '20px'
            })
        ], style={
            'display': 'flex',
            'flexDirection': 'column',
            'alignItems': 'flex-start',
            'width': '1300px'
        })
    ], style={
        'display': 'flex',
        'alignItems': 'flex-start',
        'justifyContent': 'flex-start', 
        'maxWidth': '1700px',
        'margin': '0 auto'
    }),
    html.Div([
        html.H2("Map of successful deals", style={'textAlign': 'center', 'marginBottom': '10px'}),
        dcc.Graph(figure=fig_map, style={'height': '900px', 'width': '1900px', 'margin': '0 auto'})
    ], style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center', 'marginTop': '40px'})
])

if __name__ == '__main__':
    app.run(port=8051)